In [ ]:
from iceberg.hive import HiveTables
from iceberg.api.schema import Schema
from iceberg.api.types import TimestampType, DoubleType, StringType, NestedField
from iceberg.api.partition_spec import PartitionSpecBuilder

hive_conf = {
    "hive.metastore.uris": 'thrift://hive_metastore:9083',
    "hive.metastore.warehouse.dir": "/usr/local/hadoop/warehouse"
}
tables = HiveTables(hive_conf)

table_schema = Schema(
                 NestedField.optional(1, "DateTime", TimestampType.with_timezone()),
                 NestedField.optional(2, "Bid", DoubleType.get()),
                 NestedField.optional(3, "Ask", DoubleType.get()),
                 NestedField.optional(4, "symbol", StringType.get()))

partition_spec = PartitionSpecBuilder(table_schema).add(1, 1000, "DateTime_day", "day").build()

table = tables.create(table_schema, "iceberg.test_123", partition_spec)

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.sql.warehouse.dir", "/_oreilly_iceberg/data") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hivemetastore:9083") \
    .config("spark.sql.catalog.iceberg.cache-enabled", False) \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/06 21:55:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.sql("""
    CREATE TABLE iceberg.default.sample (
        id bigint COMMENT 'unique id',
        data string
    )
    USING iceberg
""")

DataFrame[]

In [3]:
spark.sql("""
INSERT INTO iceberg.default.sample VALUES (1, 'a'), (2, 'b')
""")

23/02/06 21:57:28 ERROR Executor: Exception in task 1.0 in stage 0.0 (TID 1)
java.lang.NoClassDefFoundError: org/apache/parquet/avro/AvroWriteSupport
	at org.apache.iceberg.parquet.Parquet.write(Parquet.java:118)
	at org.apache.iceberg.parquet.Parquet$DataWriteBuilder.<init>(Parquet.java:575)
	at org.apache.iceberg.parquet.Parquet$DataWriteBuilder.<init>(Parquet.java:566)
	at org.apache.iceberg.parquet.Parquet.writeData(Parquet.java:563)
	at org.apache.iceberg.data.BaseFileWriterFactory.newDataWriter(BaseFileWriterFactory.java:121)
	at org.apache.iceberg.io.RollingDataWriter.newWriter(RollingDataWriter.java:52)
	at org.apache.iceberg.io.RollingDataWriter.newWriter(RollingDataWriter.java:32)
	at org.apache.iceberg.io.RollingFileWriter.openCurrentWriter(RollingFileWriter.java:108)
	at org.apache.iceberg.io.RollingDataWriter.<init>(RollingDataWriter.java:47)
	at org.apache.iceberg.spark.source.SparkWrite$UnpartitionedDataWriter.<init>(SparkWrite.java:694)
	at org.apache.iceberg.spark.sour

Py4JJavaError: An error occurred while calling o31.sql.
: org.apache.spark.SparkException: Writing job aborted
	at org.apache.spark.sql.errors.QueryExecutionErrors$.writingJobAbortedError(QueryExecutionErrors.scala:613)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2(WriteToDataSourceV2Exec.scala:386)
	at org.apache.spark.sql.execution.datasources.v2.V2TableWriteExec.writeWithV2$(WriteToDataSourceV2Exec.scala:330)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.writeWithV2(WriteToDataSourceV2Exec.scala:236)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run(WriteToDataSourceV2Exec.scala:309)
	at org.apache.spark.sql.execution.datasources.v2.V2ExistingTableWriteExec.run$(WriteToDataSourceV2Exec.scala:308)
	at org.apache.spark.sql.execution.datasources.v2.AppendDataExec.run(WriteToDataSourceV2Exec.scala:236)
	at org.apache.spark.sql.execution.datasources.v2.V2CommandExec.result$lzycompute(V2CommandExec.scala:43)
